In [222]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [223]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
353,tt1289406,7300000,10329747,Harry Brown,Michael Caine|Emily Mortimer|Iain Glen|Lee Oak...,Daniel Barber,Every man has a breaking point.,An elderly ex-serviceman and widower looks to ...,103,Thriller|Crime|Drama|Action,HanWay Films|UK Film Council|Marv Films,11/11/2009,6.6,2009
1131,tt0324216,9500000,107071655,The Texas Chainsaw Massacre,Jessica Biel|Jonathan Tucker|Erica Leerhsen|Mi...,Marcus Nispel,What you know about fear... doesn't even come ...,After picking up a traumatized young hitchhike...,98,Horror,New Line Cinema|Next Entertainment|Platinum Du...,10/17/2003,6.0,2003
1542,tt0433386,20000000,39143839,The Grudge 2,Sarah Michelle Gellar|Amber Tamblyn|Edison Che...,Takashi Shimizu,"What Was Once Trapped, Will Now Be Unleashed","In Tokyo, a young woman (Tamblyn) is exposed t...",102,Horror|Thriller,Columbia Pictures Corporation|Ghost House Pict...,10/13/2006,5.3,2006
878,tt0246460,140000000,431971116,Die Another Day,Pierce Brosnan|Halle Berry|Rosamund Pike|Rick ...,Lee Tamahori,Events don't get any bigger than...,The ever-daring James Bond taking on a North K...,133,Adventure|Action|Thriller,Eon Productions,11/17/2002,5.8,2002
680,tt0443701,30000000,68369434,The X Files: I Want to Believe,David Duchovny|Gillian Anderson|Amanda Peet|Bi...,Chris Carter,"To find the truth, you must believe.",Six years after the events of The X-Files seri...,104,Drama|Mystery|Science Fiction|Thriller,Twentieth Century Fox Film Corporation|Ten Thi...,7/24/2008,5.4,2008


In [224]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [225]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

In [226]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 206.7+ KB


### Check if there any non null values

In [227]:
data.isnull().values.any()

False

### Check if columns of type 'object' are of the right data format

In [228]:
object_data = data.select_dtypes(include='object').columns
object_data

Index(['imdb_id', 'original_title', 'cast', 'director', 'tagline', 'overview',
       'genres', 'production_companies', 'release_date'],
      dtype='object')

In [229]:
for ob in object_data:
    print(type(ob),'\t', ob)

<class 'str'> 	 imdb_id
<class 'str'> 	 original_title
<class 'str'> 	 cast
<class 'str'> 	 director
<class 'str'> 	 tagline
<class 'str'> 	 overview
<class 'str'> 	 genres
<class 'str'> 	 production_companies
<class 'str'> 	 release_date


### Change columns with the wrong data format to the right one

In [230]:
data['release_date'] = pd.to_datetime(data['release_date'])
type(data['release_date'][0])

pandas._libs.tslibs.timestamps.Timestamp

### Create a function collecting all answers to questions

In [231]:
answers = {}
def answer(answer, question = 1, name = True):
    if name:
        answers[question] = answer['original_title'].item() + ' (' + answer['imdb_id'].item() + ')'
    else: 
        answers[question] = answer
answers

{}

### Add profit data to the dataframe

In [232]:
data['profit'] = data['revenue'] - data['budget']
data['profit']

0       1363528810
1        228436354
2        185238201
3       1868178225
4       1316249360
           ...    
1884      82299717
1885      -8444012
1886     -40865180
1887      -3962091
1888      -9782502
Name: profit, Length: 1889, dtype: int64

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [233]:
# в словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# запишите свой вариант ответа
answers['1'] = '...'
# если ответили верно, можете добавить комментарий со значком "+"

In [234]:
# тут пишем ваш код для решения данного вопроса:

ВАРИАНТ 2

In [235]:
# можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

# 3. Какой из фильмов самый короткий (в минутах)?





# 4. Какова средняя длительность фильмов?


# 5. Каково медианное значение длительности фильмов? 

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [236]:
# лучше код получения столбца profit вынести в Предобработку что в начале

# 7. Какой фильм самый убыточный? 

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

# 9. Какой фильм оказался самым кассовым в 2008 году?

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


# 11. Какого жанра фильмов больше всего?

In [237]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

# 13. У какого режиссера самые большие суммарные кассовые сборы?

# 14. Какой режисер снял больше всего фильмов в стиле Action?

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

# 18. Самый убыточный фильм от Paramount Pictures

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [238]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '...'}

In [239]:
# и убедиться что ни чего не пропустил)
len(answers)

1